# Ising in 1D

In [ ]:
using Printf #string formatting
using Random # random number utilities
using Plots # plotting
using Statistics # statistical tools
using Distributions # useful distributions
using LaTeXStrings
using LinearAlgebra

In [ ]:
# for readability, these are good settings to use
default(xtickfontsize=14,  ytickfontsize=14, ztickfontsize=14,
    guidefontsize=14, legendfontsize=12, lw=2,ms=8)

## Setup Energy and Proposal
The energy, with $J=1$, is
$$
E(x) = - \sum_{i=1}^{N-1} \sigma_i \sigma_{i+1} - \sigma_{N}\sigma_1
$$

In [ ]:
# energy
function E(x)
    return -x[1:end-1]⋅x[2:end] -x[1] * x[end];
end

# proposal
function g(x)
    y = copy(x);
    ξ = rand(DiscreteUniform(1,length(x)));
    y[ξ] = -y[ξ];
    return y;
end

#acceptance probability
function h(x,y, β)
    return min(1, exp(β * E(x)-β* E(y))); 
end


In [ ]:
L = 64; # number of lattice sites
β = 1; # inverse temperature

X₀ = ones(Int8,L);

X = copy(X₀);
X_vals = [X];
Y = similar(X);


n = 10^4;

Random.seed!(100)
for j in 1:n-1
    Y .= g(X); 
    U = rand();
    if (U< h(X, Y, β))
        @. X = Y;
    end
    push!(X_vals, copy(X))
end


In [ ]:
# create an animation of the spin evolution
anim = @animate for i=1:10:1000
    scatter(1:L, X_vals[i],label="")
    title!(@sprintf("n = %d",i-1))
    ylims!(-1.5,1.5)
end

In [ ]:
gif(anim, fps=10)

In [ ]:
# compute energy
E_vals = [E(x) for x in X_vals];
plot(0:n-1, E_vals, label="Energy",lw=2)
plot!(0:n-1, cumsum(E_vals) ./ (1:n), label="Running Avg.",lw=2)
xlabel!(L"$t$")
title!(LaTeXString(@sprintf("\$L = %d,\\; \\beta = %g\$", L, β)))

In [ ]:
# compute mean magnetization
M_vals = [mean(x) for x in X_vals];
plot(0:n-1, M_vals, label="Magnetization",lw=2)
plot!(0:n-1, cumsum(M_vals) ./ (1:n), label="Running Avg.",lw=2)
xlabel!(L"$t$")
title!(LaTeXString(@sprintf("\$L = %d,\\; \\beta = %g\$", L, β)))

## Energy and Magnetization vs. Temperature
Compare mean energy and mean magnetization against temperature through independents runs.

In [ ]:
# energy and spin vs. temperature


L = 4; # number of lattice sites
β_vals = 10 .^(-1:0.2:1);

M_vals = Float64[];
E_vals =  Float64[];

X₀ = -ones(Int8,L);
Y = similar(X₀);

n = 10^6;

Random.seed!(100)
for β in β_vals
    X = copy(X₀);
    E_avg = E(X)/n;
    M_avg = mean(X)/n;
    for j in 1:n-1
        Y .= g(X); 
        U = rand();
        if (U< h(X, Y, β))
            @. X = Y;
        end
        E_avg += E(X)/n;
        M_avg += mean(X)/n;
    end
    
    push!(E_vals, E_avg);
    push!(M_vals, M_avg);    
    
end



In [ ]:
scatter(β_vals, E_vals, label="Numerical Estimates")
plot!(β_vals, -L * ones(length(β_vals)),label="Lower Bound", lw=2, ls=:dash)
xlabel!(L"$\beta$")
ylabel!(L"$\langle E\rangle$")

In [ ]:
scatter(β_vals, abs.(M_vals),label="Numerical Estimates")
xlabel!(L"$\beta$")
ylabel!(L"$|\langle M\rangle|$")